LOAD DATA 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

data_path = 'data/Task 2/sample_1a2bc3012c9defca.npz'


In [2]:
info = np.load(data_path, allow_pickle=True)
all_agent_trajs = info['all_agent']
all_agent_trajs.shape  #[24 agents, 91 timesteps the interval between each timestep is 0.1 second,\
#10 denotes [center_x, center_y, center_z, boundingbox_x, boundingbox_y, boundingbox_z, heading, vel_x, vel_y, valid] ]
#the valid flag is either 0 or 1. 1 means the valid, 0 means invalid
obj_types = info['object_type']
lane_polylines = info['lane']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
road_polylines = info['road_polylines']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
crosswalk_polylines = info['crosswalk']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
speed_bump_polylines = info['speed_bump']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
stop_signs_polylines = info['stop_sign']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]
drive_way_polylines = info['drive_way']  #  list of [n,7] array [x,y,z,ori_x,ori_y,ori_z,type]


We select the 11th timestep as the current timestep 

In [3]:
all_agent_current = all_agent_trajs[:, 10]
all_agent_current.shape

(24, 10)

In [4]:
#ground truth for the future 1s,2s,3s can be easily derived by :
predict_horizon = 10  #20,30
all_gt_future = all_agent_trajs[:, 11:11 + predict_horizon]
all_gt_future.shape

(24, 10, 10)

Index of target agnet to preict

In [5]:
tracks = info['predict_list']
tracks
agents_to_predict = all_agent_trajs[tracks]
agents_to_predict.shape

(8, 91, 10)

The homework should be done on these 8 agents.

Q1.Using constant velocity model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE.  

In [6]:
# constant velocity model
def calculation(gt, pred):
    ADE = np.mean(np.linalg.norm(gt - pred, axis=2), axis=1)
    FDE = np.linalg.norm(gt[:, -1] - pred[:, -1], axis=1)
    return ADE, FDE


def CVmodel(current, predict_horizon):
    num_agents = current.shape[0]
    future_states = np.zeros((num_agents, predict_horizon, 10))
    future_states[:, 0] = current
    for i in range(1, predict_horizon):
        future_states[:, i, 0] = future_states[:, i - 1, 0] + future_states[:, i - 1, 7] * 0.1
        future_states[:, i, 1] = future_states[:, i - 1, 1] + future_states[:, i - 1, 8] * 0.1
        future_states[:, i, 2] = future_states[:, i - 1, 2] + future_states[:, i - 1, 9] * 0.1
        future_states[:, i, 3:] = current[:, 3:]
    return future_states


pred= CVmodel(all_agent_current[tracks], predict_horizon)
ADEs, FDEs = calculation(all_gt_future[tracks], pred)

print('trajectories:', pred[:,:,0:3])
print('ADEs:', ADEs)
print('FDEs:', FDEs)

trajectory: [[[  693.56744385 -2454.96557617  -109.02783966]
  [  693.57897949 -2455.06884766  -108.92783966]
  [  693.59051514 -2455.17211914  -108.82783966]
  [  693.60205078 -2455.27539062  -108.72783966]
  [  693.61358643 -2455.37866211  -108.62783966]
  [  693.62512207 -2455.48193359  -108.52783966]
  [  693.63665771 -2455.58520508  -108.42783966]
  [  693.64819336 -2455.68847656  -108.32783966]
  [  693.659729   -2455.79174805  -108.22783966]
  [  693.67126465 -2455.89501953  -108.12783966]]

 [[  647.11364746 -2403.73242188  -108.92222595]
  [  647.46795654 -2403.29663086  -108.82222595]
  [  647.82226562 -2402.86083984  -108.72222595]
  [  648.17657471 -2402.42504883  -108.62222595]
  [  648.53088379 -2401.98925781  -108.52222595]
  [  648.88519287 -2401.5534668   -108.42222595]
  [  649.23950195 -2401.11767578  -108.32222595]
  [  649.59381104 -2400.68188477  -108.22222595]
  [  649.94812012 -2400.24609375  -108.12222595]
  [  650.3024292  -2399.81030273  -108.02222595]]

 [[ 

Q2.Using constant acceleration model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [7]:
# constant acceleration model
def CAmodel(current, predict_horizon):
    num_agents = current.shape[0]
    future_states = np.zeros((num_agents, predict_horizon, 10))
    future_states[:, 0] = current
    for i in range(1, predict_horizon):
        future_states[:, i, 0] = future_states[:, i - 1, 0] + future_states[:, i - 1, 7] * 0.1 + 0.5 * future_states[:,
                                                                                                       i - 1,
                                                                                                       7] * 0.1 ** 2
        future_states[:, i, 1] = future_states[:, i - 1, 1] + future_states[:, i - 1, 8] * 0.1 + 0.5 * future_states[:,
                                                                                                       i - 1,
                                                                                                       8] * 0.1 ** 2
        future_states[:, i, 2] = future_states[:, i - 1, 2] + future_states[:, i - 1, 9] * 0.1 + 0.5 * future_states[:,
                                                                                                       i - 1,
                                                                                                       9] * 0.1 ** 2
        future_states[:, i, 3:] = current[:, 3:]
    return future_states


pred = CAmodel(all_agent_current[tracks], predict_horizon)
ADEs, FDEs = calculation(all_gt_future[tracks], pred)

print('trajectories:', pred[:,:,0:3])
print('ADEs:', ADEs)
print('FDEs:', FDEs)

trajectory: [[[  693.56744385 -2454.96557617  -109.02783966]
  [  693.57955627 -2455.07401123  -108.92283966]
  [  693.5916687  -2455.18244629  -108.81783966]
  [  693.60378113 -2455.29088135  -108.71283966]
  [  693.61589355 -2455.39931641  -108.60783966]
  [  693.62800598 -2455.50775146  -108.50283966]
  [  693.64011841 -2455.61618652  -108.39783966]
  [  693.65223083 -2455.72462158  -108.29283966]
  [  693.66434326 -2455.83305664  -108.18783966]
  [  693.67645569 -2455.9414917   -108.08283966]]

 [[  647.11364746 -2403.73242188  -108.92222595]
  [  647.485672   -2403.27484131  -108.81722595]
  [  647.85769653 -2402.81726074  -108.71222595]
  [  648.22972107 -2402.35968018  -108.60722595]
  [  648.60174561 -2401.90209961  -108.50222595]
  [  648.97377014 -2401.44451904  -108.39722595]
  [  649.34579468 -2400.98693848  -108.29222595]
  [  649.71781921 -2400.52935791  -108.18722595]
  [  650.08984375 -2400.07177734  -108.08222595]
  [  650.46186829 -2399.61419678  -107.97722595]]

 [[ 

Using Constant Turn Rate and Velocity(CTRV) model to predict the future (1s,2s,3s) trajectories of the target agent and calculate the ADE and FDE. 

In [8]:
# CTRV model
def CTRVmodel(current_state, predict_horizon, omega):
    # Assuming the turn rate omega is given for each agent
    num_agents = current_state.shape[0]
    future_states = np.zeros((num_agents, predict_horizon, 10))
    dt = 0.1  # Time step in seconds
    for agent in range(num_agents):
        # Initialize the state for the first prediction horizon
        future_states[agent, 0, :] = current_state[agent]
        for i in range(1, predict_horizon):
            theta = future_states[agent, i - 1, 6]  # Current heading
            v = np.sqrt(current_state[agent, 7] ** 2 + current_state[agent, 8] ** 2)  # Velocity magnitude
            # Check if the turn rate is effectively zero
            if np.abs(omega[agent]) < 1e-5:  # Consider as straight motion
                dx = v * np.cos(theta) * dt
                dy = v * np.sin(theta) * dt
            else:  # Turning motion
                dx = (v / omega[agent]) * (np.sin(theta + omega[agent] * dt) - np.sin(theta))
                dy = (v / omega[agent]) * (-np.cos(theta + omega[agent] * dt) + np.cos(theta))
            # Update position
            future_states[agent, i, 0] = future_states[agent, i - 1, 0] + dx
            future_states[agent, i, 1] = future_states[agent, i - 1, 1] + dy
            # Assuming Z, bounding box, and other parameters remain constant for simplicity
            future_states[agent, i, 2:9] = current_state[agent, 2:9]
            # Update heading
            future_states[agent, i, 6] = theta + omega[agent] * dt

    return future_states


omega = np.zeros(all_agent_current.shape[0])  # Assuming zero turn rate for all agents
pred = CTRVmodel(all_agent_current[tracks], predict_horizon, omega)
ADEs, FDEs = calculation(all_gt_future[tracks], pred)

print('trajectories:', pred[:,:,0:3])
print('ADEs:', ADEs)
print('FDEs:', FDEs)

trajectory: [[[  693.56744385 -2454.96557617  -109.02783966]
  [  693.56790496 -2455.06948891  -109.02783966]
  [  693.56836607 -2455.17340166  -109.02783966]
  [  693.56882718 -2455.2773144   -109.02783966]
  [  693.56928829 -2455.38122714  -109.02783966]
  [  693.56974941 -2455.48513988  -109.02783966]
  [  693.57021052 -2455.58905262  -109.02783966]
  [  693.57067163 -2455.69296537  -109.02783966]
  [  693.57113274 -2455.79687811  -109.02783966]
  [  693.57159385 -2455.90079085  -109.02783966]]

 [[  647.11364746 -2403.73242188  -108.92222595]
  [  647.46702444 -2403.29587469  -108.92222595]
  [  647.82040142 -2402.8593275   -108.92222595]
  [  648.1737784  -2402.42278032  -108.92222595]
  [  648.52715538 -2401.98623313  -108.92222595]
  [  648.88053236 -2401.54968595  -108.92222595]
  [  649.23390934 -2401.11313876  -108.92222595]
  [  649.58728632 -2400.67659157  -108.92222595]
  [  649.94066329 -2400.24004439  -108.92222595]
  [  650.29404027 -2399.8034972   -108.92222595]]

 [[ 

Briefly discuss the observations from your results. 

If you want to visualize the scenario here are some simple demo codes for you.

    'TYPE_FREEWAY': 1,
    'TYPE_SURFACE_STREET': 2,
    'TYPE_BIKE_LANE': 3,

    # for roadline
    'TYPE_UNKNOWN': -1,
    'TYPE_BROKEN_SINGLE_WHITE': 6,
    'TYPE_SOLID_SINGLE_WHITE': 7,
    'TYPE_SOLID_DOUBLE_WHITE': 8,
    'TYPE_BROKEN_SINGLE_YELLOW': 9,
    'TYPE_BROKEN_DOUBLE_YELLOW': 10,
    'TYPE_SOLID_SINGLE_YELLOW': 11,
    'TYPE_SOLID_DOUBLE_YELLOW': 12,
    'TYPE_PASSING_DOUBLE_YELLOW': 13,

    # for roadedge
    'TYPE_ROAD_EDGE_BOUNDARY': 15,
    'TYPE_ROAD_EDGE_MEDIAN': 16,

    # for stopsign
    'TYPE_STOP_SIGN': 17,

    # for crosswalk
    'TYPE_CROSSWALK': 18,

    # for speed bump
    'TYPE_SPEED_BUMP': 19,
    
    # for driveway
    'TYPE_DRIVEWAY': 20,

In [9]:
sdc_current_state = all_agent_trajs[tracks[0]][11]
ax = plt.gca()
fig = plt.gcf()
fig.set_facecolor('xkcd:grey')
ax.set_facecolor('xkcd:grey')
for polyline in road_polylines:
    map_type = polyline[0, 6]
    if map_type == 6:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='dashed', linewidth=1)
    elif map_type == 7:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 8:
        plt.plot(polyline[:, 0], polyline[:, 1], 'w', linestyle='solid', linewidth=1)
    elif map_type == 9:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 10:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dashed', linewidth=1)
    elif map_type == 11:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 12:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='solid', linewidth=1)
    elif map_type == 13:
        plt.plot(polyline[:, 0], polyline[:, 1], 'xkcd:yellow', linestyle='dotted', linewidth=1)
    elif map_type == 15:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
    elif map_type == 16:
        plt.plot(polyline[:, 0], polyline[:, 1], 'k', linewidth=1)
ax.axis([-70 + sdc_current_state[0], 70 + sdc_current_state[0], -70 + sdc_current_state[1], 70 + sdc_current_state[1]])

filename = './viz.png'
plt.savefig(filename)
plt.close()